# Custom view report per user

In [2]:
from arcgis.gis import GIS
from getpass import getpass
import pandas as pd
from pprint import pprint

In [3]:
gis = GIS('https://www.arcgis.com', 'tkinlaw', getpass())

## Search for content
The default `max_items` is 10. Be sure to change that. Sorting can be done with `sort_field` and `sort_order`. These rely on basic fields from the item. View is not one of them. 
We want views. This is a property of the item, and is not exposed to `search` or `advanced_search`

In [4]:
allContent = gis.content.search('', max_items=500, sort_field = 'type', sort_order="desc")
print(len(allContent))

205


c:\Users\tim10393\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinlaw.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\tim10393\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinlaw.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\tim10393\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinlaw.maps.arcgis.com'. Adding certific

## Bring results into a DataFrame for sorting

### Sort all content by views

In [5]:
rows = []
for item in allContent:
    rows.append({
        "title": item.title,
        "item": item,
        "type": item.type,
        "numViews": item.numViews,
        "user": item.owner
    })
df = pd.DataFrame(rows)
df.sort_values(by="numViews", ascending=False).head(10)

,title,item,type,numViews,user
49,Analysis,"{'id': '05407f2d7dce47a0bd89641e00a53511', 'ow...",StoryMap Theme,882,tkinlaw
157,Aircraft_WFL1,"{'id': '2d20dc78add14bad8dc2a0d4d8ae5f40', 'ow...",Feature Service,427,tkinlaw
52,Coaching Day Logistics,"{'id': '5e1c4ffe2c564606a0eb8c188f0b4c8c', 'ow...",StoryMap,424,tkinlaw
55,Training the model,"{'id': '2c327ec1ba6d4534b518aec53b593b8a', 'ow...",StoryMap,382,tkinlaw
56,Exporting training samples,"{'id': '0e59c289163f4895999540c3338b94ad', 'ow...",StoryMap,330,tkinlaw
21,Aircraft,"{'id': '24be629489304834bcdef018b4312da3', 'ow...",Web Map,302,tkinlaw
51,Preparing imagery for use,"{'id': '61c161ab92a74887870327fd53cdb068', 'ow...",StoryMap,295,tkinlaw
48,KNOW ramp plan,"{'id': '5acb2e9d636940a7bdc964fe52cf57fb', 'ow...",StoryMap Theme,263,tkinlaw
54,Deep learning,"{'id': '4dc982736eb1451783b825c810c3077b', 'ow...",StoryMap,251,tkinlaw
57,Building detection exercise alternatives,"{'id': 'df7ce715b1c94e6e98029244f210cef3', 'ow...",StoryMap,248,tkinlaw


### Sort user content by views

In [10]:
userName = 'JohnDoe748'
df[df['user']==userName].sort_values(by="numViews", ascending=False).head(10)

,title,item,type,numViews,user
177,S_USA_Wilderness,"{'id': 'cd3c72e2b7fc4458ab4814fc4933b2a0', 'ow...",Feature Service,17,JohnDoe748
184,Bigfoot_Sightings,"{'id': '2e71e69b7a1342d3911ea7ba64e710c4', 'ow...",Feature Service,13,JohnDoe748
61,S_USA_Wilderness,"{'id': '05518289df9a4776970e3f6ff089a8da', 'ow...",Shapefile,1,JohnDoe748
40,US Wilderness Areas,"{'id': '2d3eb177ff7c401193a3959f3318a4d8', 'ow...",Web Map,1,JohnDoe748
62,Bigfoot_Sightings,"{'id': 'a4893d3c358846edadf73bcc593e46c0', 'ow...",Shapefile,1,JohnDoe748
135,S_USA_AdministrativeRegion_gdb,"{'id': 'a75a720d6ac04803a9c89ed2d3a6f8a9', 'ow...",File Geodatabase,0,JohnDoe748


In [13]:
gis.users.search('')

[<User username:JaneDoe0>,
 <User username:JohnDoe748>,
 <User username:mhnudi>,
 <User username:Tester0>,
 <User username:tkinlaw>]

In [19]:
gis.users.properties

AttributeError: 'UserManager' object has no attribute 'properties'